# Helium-Atom

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Grundzustandsenergie des Helium-Atoms im Rahmen der Hartree-Näherung berechnet. Für die numerische Rechnung ist es günstig, statt der Wellenfunktion $\psi(r)$ die Funktion

$$f(r) = \sqrt{4 \pi}r\psi(r)$$

zu verwenden, die der Hartree-Gleichung 

$$\left( \frac{1}{2} \frac{\text{d}^2}{\text{d} r^2} - \frac{2}{r} + V_\text{H}(r) \right) f(r) = \epsilon f(r)$$

genügen soll. Dabei ist das Hartree-Potential durch

$$V_\text{H}(r) = -\int\limits_r^\infty\text{d}r' \frac{q(r')}{r^2}$$

mit

$$q(r') = - \int \limits_0^r' f^2(r'') \, \text{d}r''$$

gegeben. Da das Hartree-Potential von $f(r)$ abhängt, lösen wir die Hartree-Gleichung iterativ. Mit der konvergierten Lösung für $f(r)$ erhält man die Grundzustandsenergie als

$$E = 2\epsilon - \int_0^\infty\text{d}r V_\text{H}(r)f^2(r)\,.$$

## Importanweisungen

In diesem Jupyter-Notebook importieren wir zwei neue Module aus dem SciPy-Paket. Das `constants`-Modul enthält Informationen über physikalische Konstanten, die z.B. mit Hilfe der Funktion `physical_constant` erhalten werden können. Auf diese Weise werden wir uns am Ende den Referenzwert für die Hartree-Energie beschaffen. Außerdem verwenden wir das `sparse`-Modul und das zugehörige `linalg`-Modul, um effizient mit dünnbesetzten Matrizen arbeiten zu können.

In [ ]:
from math import pi, sqrt
import numpy as np
from scipy.constants import physical_constants
from scipy import sparse
from scipy.sparse import linalg
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Berechnung des Hartree-Potentials

Das Hartree-Potential wird für eine Funktion $f(r)$, die auf dem Ortsgitter `r` mit der Schrittweite `dr` im NumPy-Array `f` übergeben wird, mittels der oben angegebenen Formeln für $V_\text{H}(r)$ und $q(r)$ berechnet. Um die Integrationsergebnisse auf dem gesamten Ortsgitter zu erhalten, berechnen wir unter Verwendung der Funktion `cumsum` aus dem NumPy-Paket kumulative Summen.

Da wir nur auf einem endlich großen Gitter rechnen, das bis zu einem Abstand $r_\text{max}$ vom Atomkern reicht, können wir das Integral für das Hartree-Potential nicht bis unendlich berechnen. Um dennoch sicherzustellen, dass das Potential im Unendlichen verschwindet, nehmen wir an, dass sich die gesamte Ladungsverteilung innerhalb der Kugel mit Radius $r_\text{max}$ befindet und das Hartree-Potential außerhalb deswegen durch 1/r gegeben ist. Deswegen wird vom berechneten Hartree-Potential der Wert beim maximalen Radius abgezogen und dafür der Wert $1/r_\text{max}$ hinzugezählt.

Als Ergebnis wird nicht nur das Hartree-Potential zurückgegeben, sondern auch die effektive Ladung $q_\text{eff}(r)$, die das Elektron sieht und die sich aus der positiven Ladung des Kerns mit der Kernladungszahl 2 und der abschirmenden Ladung des anderen Elektrons zusammensetzt.

In [ ]:
def h_potential(r, dr, f):
    q = -np.cumsum(f*f) * dr
    hartree_potential = np.cumsum(q/r**2) * dr
    correction = -hartree_potential[-1] + 1/r[-1]
    hartree_potential = hartree_potential + correction
    return hartree_potential, 2+q

## Berechnung des Hamilton-Operators

Die diskretisierte Version des Hamilton-Operators wird als dünnbesetzte Matrix definiert. Dazu diskretisieren wir den Operator der kinetischen Energie wie in [5-05-Freies-Teilchen.ipynb](5-05-Freies-Teilchen.ipynb), verwenden dann aber Funktionen aus dem SciPy-Modul `sparse`. Außerdem sind das Potential des Atomkerns und das Hartree-Potential zu berücksichtigen. 

In [ ]:
def hamilton_operator(r, dr, f):
    n_max = r.shape[0]
    h_kin = (sparse.eye(n_max)
             - 0.5*(sparse.eye(n_max, k=1)
                    + sparse.eye(n_max, k=-1))
             )/dr**2
    hartree_potential, q_eff = h_potential(r, dr, f)
    h = h_kin + sparse.diags(-2/r + hartree_potential)
    return h

## Berechnung der Eigenwerte und Eigenvektoren

Für die Hamilton-Matrix `h` werden die Eigenwerte `ew` und Eigenvektoren `ev` mit Hilfe der Funktion `sparse.linalg.eigsh` aus dem SciPy-Paket bestimmt, die für hermische Matrizen geeignet ist. Diese Funktion für dünnbesetzte Matrizen ist nicht geeignet, um das gesamte Spektrum zu berechnen. Da wir uns ohnehin nur für den Grundzustand, also einen einzigen Zustand, interessieren, setzen wir das Argument `k` auf 1. Um den richtigen Eigenzustand zu erwischen, geben wir mit dem Argument `sigma` an, dass der Eigenwert in der Nähe von -2 liegen soll.

In [ ]:
def get_groundstate(r, dr, f):
    h = hamilton_operator(r, dr, f)
    ew, ev = sparse.linalg.eigsh(h, k=1, sigma=-2)
    return ew[0], ev[:, 0]/sqrt(dr)

## Berechnung der Grundzustandsenergie

Zur Auswertung der eingangs angegebenen Formel für $E$ nähern wir das Integral durch eine Summe über den Integranden auf dem verwendeten radialen Gitter.

In [ ]:
def energy(r, dr, f, epsilon):
    hartree_potential, q_eff = h_potential(r, dr, f)
    e = 2*epsilon - sum(hartree_potential*f*f) * dr
    return e

## Berechnung der selbstkonsistenten Lösung durch Iteration

Zur iterativen Bestimmung der selbstkonsistenten Lösung setzen wir die Funktion $f(r)$ in der Variablen `f` anfänglich zu null und führen dann Iterationsschritte durch Aufruf der Funktion `get_groundstate` durch. Dabei ergibt sich ein neues NumPy-Array `f`, das dann in die nächste Iteration eingeht. Die Berechnung wird beendet, wenn entweder die relative Änderung der Energie $\epsilon$ kleiner als die vorgegebene Genauigkeit `delta` ist oder wenn die maximale Zahl an Iterationen `n_iter_max` erreicht wurde. Danach wird aus der Einelektronenenergie `epsilon` die Grundzustandsenergie `e` bestimmt und diese zusammen mit der Funktion `f` und der Anzahl der Iteration zurückgegeben.

In [ ]:
def hartree_iterations(r, dr, n_iter_max, delta):
    f = np.zeros_like(r)
    epsilon_old = 0
    for n_iter in range(n_iter_max):
        epsilon, f = get_groundstate(r, dr, f)
        if abs((epsilon-epsilon_old)/epsilon) < delta:
            break
        epsilon_old = epsilon
    e = energy(r, dr, f, epsilon)
    return e, f, n_iter+1

## Implementierung der Bedienelemente und Ausgabe der Ergebnisse

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `r_max`: maximaler Radius
- `n_max`: Anzahl der zu verwendenden Radiuswerte ohne den Ursprung
- `n_iter_max`: maximale Anzahl der Iterationen

Nach Ausführung des Hartree-Verfahrens bis zu der zu Beginn der Funktion `plot_result` festgelegten Genauigkeit werden die Anzahl der Iterationen, die numerisch bestimmte Grundzustandsenergie sowie zum Vergleich der experimentell ermittelte Wert ausgegeben. In den Diagrammen werden die Wellenfunktion $\psi(r)$, die damit zusammenhängende Funktion $f(r)$, das Hartree-Potential $V_\text{H}(r)$ sowie die effektive Kernladung $q_\text{eff}(r)$ als Funktion des Abstands vom Atomkern dargestellt. Im letzten Diagramm ist deutlich die Abschirmung der Kerns durch das andere Elektron zu erkennen.

In [ ]:
widget_dict = {"r_max":
               widgets.FloatSlider(
                   value=30, min=5, max=100, step=5,
                   description=r"$r_\text{max}$"),
               "n_max":
               widgets.IntSlider(
                   value=10000, min=1000, max=20000,
                   description=r"$n_\text{max}$"),
               "n_iter_max":
               widgets.IntSlider(
                   value=30, min=10, max=100, step=10,
                   description=r"$n_\text{iter max}$")
               }

@interact(**widget_dict)
def plot_result(r_max, n_max, n_iter_max):
    delta = 1e-6
    r_plot_max = 10
    r_all, dr = np.linspace(0, r_max, n_max+1, retstep=True)
    r = r_all[1:]
    e, f, n_iter = hartree_iterations(
        r, dr, n_iter_max, delta)
    print(f"{n_iter} Iterationen")
    e_h = physical_constants["Hartree energy in eV"][0]
    print(f"Numerischer Wert    : {e*e_h:6.3f} eV")
    print("Experimenteller Wert: -79.005 eV")
    ground_state = f/(sqrt(4*pi)*r)
    hartree_potential, q_eff = h_potential(r, dr, f)
    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(r, ground_state)
    axs[0, 0].set_ylabel(r"$\psi(r)$")
    axs[0, 0].set_xlim((0, r_plot_max))
    axs[0, 1].plot(r, f)
    axs[0, 1].set_ylabel("$f(r)$")
    axs[0, 1].set_xlim((0, r_plot_max))
    axs[1, 0].plot(r, hartree_potential)
    axs[1, 0].set_xlabel("$r$")
    axs[1, 0].set_ylabel(r"$V_\mathrm{H}(r)$")
    axs[1, 0].set_xlim((0, r_plot_max))
    axs[1, 1].plot(r, q_eff)
    axs[1, 1].set_xlabel("$r$")
    axs[1, 1].set_ylabel(r"$q_\mathrm{eff}(r)$")
    axs[1, 1].set_xlim((0, r_plot_max))